In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets-classificationsynopsis/final_combined_movies_5genres.csv


In [2]:
# -*- coding: utf-8 -*-
"""
Movie Genre Classification with IndoBERT
Environment: Kaggle
"""

!pip install optuna


In [3]:
# BAGIAN PERTAMA - Import dan Konfigurasi
import os
import logging
import datetime
import json
import argparse
import gc
import sys
import codecs
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union

import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import psutil
import optuna
from optuna.trial import Trial

# Define Base Path for Kaggle
BASE_PATH = Path('/kaggle/working')
DATASETS_PATH = Path('/kaggle/input/datasets-classificationsynopsis')

# Configuration Constants
class Config:
    # Model Parameters
    MODEL_PARAMS = {
        'EPOCHS': 100,
        'BATCH_SIZE': 10,
        'LEARNING_RATE': 1e-5,
        'MAX_LENGTH': 512,
        'TEST_SIZE': 0.15,
        'WEIGHT_DECAY': 0.05,
        'MIXUP_PROB': 0.5,
        'PATIENCE': 5,
        'SMOOTHING': 0.2
    }

    # Optimization Parameters
    OPTIM_PARAMS = {
        'batch_size': [2, 4, 8],
        'learning_rate': [3e-6, 5e-6, 8e-6],
        'weight_decay': [0.01, 0.02],
        'mixup_prob': [0.2, 0.3],
        'smoothing': [0.1, 0.15]
    }

    # Paths Configuration untuk Kaggle
    BASE_DIR = BASE_PATH
    DATA_PATH = Path('/kaggle/input/datasets-classificationsynopsis/final_combined_movies_5genres.csv')
    LOG_DIR = BASE_DIR / 'logs'
    BACKUP_DIR = BASE_DIR / 'backups'
    TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    EXPERIMENT_DIR = LOG_DIR / 'experiments' / TIMESTAMP

    # Model and Data Paths
    MODEL_SAVE_DIR = EXPERIMENT_DIR / 'model'
    TOKENIZER_SAVE_DIR = EXPERIMENT_DIR / 'tokenizer'
    METRICS_DIR = EXPERIMENT_DIR / 'metrics'
    PLOTS_DIR = EXPERIMENT_DIR / 'plots'
    CM_DIR = PLOTS_DIR / 'confusion_matrices'

    # Model Files
    MODEL_BEST_ACC = MODEL_SAVE_DIR / "best_accuracy"
    MODEL_BEST_LOSS = MODEL_SAVE_DIR / "best_loss"
    TOKENIZER_BEST_ACC = TOKENIZER_SAVE_DIR / "best_accuracy"
    TOKENIZER_BEST_LOSS = TOKENIZER_SAVE_DIR / "best_loss"
    DATA_PATH = Path('/kaggle/input/datasets-classificationsynopsis/final_combined_movies_5genres.csv')  # Path langsung ke file CSV

    # Device Configuration
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SAMPLE_SIZE: Optional[int] = None

    @classmethod
    def create_directories(cls) -> None:
        """Create all necessary directories in Kaggle working directory"""
        directories = [
            cls.LOG_DIR, cls.BACKUP_DIR,
            cls.EXPERIMENT_DIR, cls.MODEL_SAVE_DIR, cls.TOKENIZER_SAVE_DIR,
            cls.METRICS_DIR, cls.PLOTS_DIR, cls.CM_DIR
        ]
        for dir_path in directories:
            dir_path.mkdir(parents=True, exist_ok=True)
            print(f"Created directory: {dir_path}")

    @classmethod
    def setup_logging(cls) -> None:
        """Setup logging configuration untuk Kaggle"""
        log_file = cls.EXPERIMENT_DIR / 'training.log'
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file, encoding='utf-8', mode='a'),
                logging.StreamHandler(sys.stdout)
            ]
        )
        logging.info(f"Log file created at: {log_file}")

# Environment Check Function
def check_environment() -> bool:
    """Verify Kaggle environment and paths"""
    try:
        # Check if datasets directory exists
        if not DATASETS_PATH.exists():
            raise RuntimeError(f"Dataset directory tidak ditemukan di: {DATASETS_PATH}")
        
        # List available files in dataset directory
        print("\nFiles in dataset directory:")
        for file in DATASETS_PATH.glob('*'):
            print(f"- {file.name}")

        # Check if dataset exists
        if not Config.DATA_PATH.exists():
            raise RuntimeError(f"Dataset tidak ditemukan di: {Config.DATA_PATH}")

        # Check GPU availability
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert to GB
            print(f"GPU tersedia: {gpu_name}")
            print(f"GPU Memory: {gpu_memory:.2f} GB")
        else:
            print("WARNING: GPU tidak tersedia, menggunakan CPU")

        return True
    except Exception as e:
        print(f"Error dalam setup environment: {str(e)}")
        return False

# Memory Management
class ModelManager:
    """Context manager for model memory management"""
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __enter__(self):
        return self.model, self.tokenizer

    def __exit__(self, exc_type, exc_val, exc_tb):
        del self.model
        del self.tokenizer
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

# Error Handling
def error_handler(func):
    """Decorator for consistent error handling"""
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logging.error(f"Error in {func.__name__}: {str(e)}")
            raise
    return wrapper

# Utility Functions
def get_memory_usage() -> float:
    """Get current memory usage of the program"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # in MB

def log_memory(step_name: str) -> None:
    """Log memory usage with consistent format"""
    memory = get_memory_usage()
    logging.info(f"Memory usage after {step_name}: {memory:.2f} MB")

def log_system_info() -> None:
    """Log system information including GPU details"""
    logging.info("System Information:")
    logging.info(f"Python Version: {sys.version}")
    logging.info(f"CPU Count: {os.cpu_count()}")
    logging.info(f"Initial Memory Usage: {get_memory_usage():.2f} MB")
    if torch.cuda.is_available():
        logging.info(f"GPU Device: {torch.cuda.get_device_name(0)}")
        logging.info(f"GPU Memory Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        logging.info(f"CUDA Version: {torch.version.cuda}")

# Create directories and setup logging
if check_environment():
    Config.create_directories()
    Config.setup_logging()
    log_system_info()
else:
    print("Failed to initialize environment. Please check the setup.")
    sys.exit(1)


Files in dataset directory:
- final_combined_movies_5genres.csv
GPU tersedia: Tesla T4
GPU Memory: 15.83 GB
Created directory: /kaggle/working/logs
Created directory: /kaggle/working/backups
Created directory: /kaggle/working/logs/experiments/20250219_084004
Created directory: /kaggle/working/logs/experiments/20250219_084004/model
Created directory: /kaggle/working/logs/experiments/20250219_084004/tokenizer
Created directory: /kaggle/working/logs/experiments/20250219_084004/metrics
Created directory: /kaggle/working/logs/experiments/20250219_084004/plots
Created directory: /kaggle/working/logs/experiments/20250219_084004/plots/confusion_matrices
2025-02-19 08:40:04,475 - INFO - Log file created at: /kaggle/working/logs/experiments/20250219_084004/training.log
2025-02-19 08:40:04,475 - INFO - System Information:
2025-02-19 08:40:04,476 - INFO - Python Version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
2025-02-19 08:40:04,482 - INFO - CPU Count: 4
2025-02-19 08:40:04,484 - INFO

In [4]:
# BAGIAN KEDUA - Dataset dan Data Processing

class MovieDataset(Dataset):
    """Dataset class untuk movie genre classification"""
    def __init__(self, texts: Union[List, np.ndarray],
                 labels: Union[List, np.ndarray],
                 tokenizer,
                 max_length: int = 512):
        # Input validation
        if not isinstance(texts, (list, np.ndarray)):
            raise ValueError("texts must be a list or numpy array")
        if not isinstance(labels, (list, np.ndarray)):
            raise ValueError("labels must be a list or numpy array")
        if len(texts) != len(labels):
            raise ValueError("texts and labels must have the same length")

        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        return len(self.texts)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten().long(),
            'attention_mask': encoding['attention_mask'].flatten().long(),
            'labels': torch.FloatTensor(self.labels[idx])
        }

class DataProcessor:
    """Class for handling data preprocessing and loading"""
    @staticmethod
    def clean_text(text: str) -> str:
        """Clean and preprocess text data"""
        if isinstance(text, str):
            text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
            text = re.sub(r'\S+@\S+', '', text)
            text = re.sub(r'[^\w\s]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            return text.strip().lower()
        return ''

    @staticmethod
    @error_handler
    def load_and_preprocess_data(data_path: Path, sample_size: Optional[int] = None) -> pd.DataFrame:
        """Load and preprocess data with proper encoding handling"""
        if not data_path.exists():
            raise FileNotFoundError(f"Data file not found: {data_path}")

        if sample_size is not None and (not isinstance(sample_size, int) or sample_size <= 0):
            raise ValueError("sample_size must be a positive integer")

        logging.info("Loading and preprocessing data...")
        log_memory("start")
        initial_size = None

        # Try different encodings for Google Drive compatibility
        encodings_to_try = ['utf-8', 'utf-8-sig', 'latin1', 'iso-8859-1', 'cp1252']
        df = None

        for encoding in encodings_to_try:
            try:
                df = pd.read_csv(data_path, encoding=encoding)
                logging.info(f"Successfully loaded data using {encoding} encoding")
                initial_size = len(df)
                break
            except (UnicodeDecodeError, UnicodeError):
                continue

        if df is None:
            raise UnicodeError(f"Failed to read file with any of these encodings: {encodings_to_try}")

        log_memory("data loading")

        # Sample data if requested
        if sample_size:
            if sample_size > initial_size:
                logging.warning(f"Requested sample_size ({sample_size}) is larger than dataset size ({initial_size})")
                sample_size = initial_size
            logging.info(f"Taking sample of {sample_size} from {initial_size} total samples")
            df = df.head(sample_size)
        else:
            logging.info(f"Using full dataset with {initial_size} samples")

        # Log sample data
        logging.info("\nSample data:")
        for i, row in df.head(3).iterrows():
            logging.info(f"\nSample {i+1}:")
            logging.info(f"Synopsis: {row['sinopsis'][:100]}...")
            logging.info(f"Genre: {row['genre']}")

        # Preprocess data
        logging.info("\nPreprocessing text data...")
        tqdm.pandas()
        df['sinopsis'] = df['sinopsis'].progress_apply(DataProcessor.clean_text)
        df['genre'] = df['genre'].str.split(',')
        df = df.dropna(subset=['sinopsis', 'genre'])

        log_memory("preprocessing")
        return df

    @staticmethod
    def prepare_data(df: pd.DataFrame, mlb: MultiLabelBinarizer) -> Tuple:
        """Prepare data for training"""
        genre_labels = mlb.fit_transform(df['genre'])
        return train_test_split(
            df['sinopsis'].values,
            genre_labels,
            test_size=Config.MODEL_PARAMS['TEST_SIZE'],
            random_state=42,
            stratify=genre_labels if len(genre_labels.shape) == 1 else None
        )

    @staticmethod
    def create_weighted_sampler(genre_labels: np.ndarray) -> WeightedRandomSampler:
        """Create weighted sampler for balanced batch sampling"""
        logging.info("Creating weighted sampler for balanced batch sampling...")

        sample_weights = np.zeros(len(genre_labels))
        for i in range(genre_labels.shape[1]):
            sample_weights += genre_labels[:, i] * (1.0 / np.sum(genre_labels[:, i]))

        sample_weights = sample_weights / sample_weights.sum()
        sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )

        logging.info(f"Created sampler with {len(sample_weights)} weights")
        return sampler

    @staticmethod
    def calculate_class_weights(genre_labels: np.ndarray, mlb: MultiLabelBinarizer) -> torch.Tensor:
        """Calculate class weights for handling imbalanced data"""
        class_weights = []
        logging.info("\nCalculating class weights for handling imbalanced data...")

        for i in range(genre_labels.shape[1]):
            genre = mlb.classes_[i]
            positive_samples = np.sum(genre_labels[:, i])
            total_samples = len(genre_labels)

            weights = compute_class_weight(
                class_weight='balanced',
                classes=np.array([0, 1]),
                y=genre_labels[:, i]
            )
            class_weights.append(weights[1])

            logging.info(f"{genre}:")
            logging.info(f"  Positive samples: {positive_samples}")
            logging.info(f"  Negative samples: {total_samples - positive_samples}")
            logging.info(f"  Weight: {weights[1]:.2f}")

        return torch.FloatTensor(class_weights).to(Config.DEVICE)

class ModelSetup:
    """Class for handling model setup and data loaders"""
    @staticmethod
    def setup_model_and_tokenizer(num_labels: int) -> Tuple:
        """Setup model dan tokenizer"""
        logging.info("Setting up model and tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
        model = AutoModelForSequenceClassification.from_pretrained(
            'indobenchmark/indobert-base-p1',
            num_labels=num_labels,
            problem_type="multi_label_classification"
        ).to(Config.DEVICE)
        logging.info("Model and tokenizer setup completed")
        return model, tokenizer

    @staticmethod
    def setup_dataloaders(X_train: np.ndarray,
                         X_test: np.ndarray,
                         y_train: np.ndarray,
                         y_test: np.ndarray,
                         tokenizer,
                         batch_size: int) -> Tuple:
        """Setup data loaders"""
        logging.info("Setting up data loaders...")
        train_dataset = MovieDataset(X_train, y_train, tokenizer)
        val_dataset = MovieDataset(X_test, y_test, tokenizer)
        sampler = DataProcessor.create_weighted_sampler(y_train)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=sampler,
            num_workers=0,  # Set to 0 for Colab compatibility
            pin_memory=True if torch.cuda.is_available() else False
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            num_workers=0,  # Set to 0 for Colab compatibility
            pin_memory=True if torch.cuda.is_available() else False
        )

        logging.info(f"Created data loaders with batch size {batch_size}")
        return train_loader, val_loader

In [5]:
# BAGIAN KETIGA - Loss Functions dan Training

class LossFunctions:
    """Class untuk menangani berbagai loss functions"""

    @staticmethod
    def focal_loss(predictions: torch.Tensor,
                  targets: torch.Tensor,
                  gamma: float = 2.0,
                  alpha: float = 0.25) -> torch.Tensor:
        """Calculate focal loss for multi-label classification"""
        ce_loss = F.binary_cross_entropy_with_logits(predictions, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = alpha * (1-pt)**gamma * ce_loss
        return focal_loss.mean()

    @staticmethod
    def label_smoothing_loss(outputs: torch.Tensor,
                           targets: torch.Tensor,
                           smoothing: float) -> torch.Tensor:
        """Calculate loss with label smoothing"""
        log_probs = F.log_softmax(outputs, dim=-1)
        targets = torch.clamp(targets * (1.0 - smoothing), min=smoothing / (targets.size(-1) - 1))
        return torch.mean(torch.sum(-targets * log_probs, dim=-1))

class DataAugmentation:
    """Class untuk menangani augmentasi data"""

    @staticmethod
    def apply_mixup(batch: Dict[str, torch.Tensor], alpha: float = 0.2) -> Dict[str, torch.Tensor]:
        """Apply mixup augmentation to batch"""
        # Move tensors to device
        input_ids = batch['input_ids'].to(Config.DEVICE)
        attention_mask = batch['attention_mask'].to(Config.DEVICE)
        labels = batch['labels'].to(Config.DEVICE)

        lam = np.random.beta(alpha, alpha)
        mixed_input_ids = lam * input_ids + (1 - lam) * input_ids.flip(0)
        mixed_attention_mask = lam * attention_mask + (1 - lam) * attention_mask.flip(0)
        mixed_labels = lam * labels + (1 - lam) * labels.flip(0)

        return {
            'input_ids': mixed_input_ids.long(),
            'attention_mask': mixed_attention_mask.long(),
            'labels': mixed_labels
        }

class Visualization:
    """Class untuk menangani visualisasi"""

    @staticmethod
    def plot_confusion_matrices(predictions: np.ndarray,
                              labels: np.ndarray,
                              classes: List[str]) -> None:
        """Plot detailed confusion matrices for each genre"""
        logging.info("Generating detailed confusion matrices for each genre...")

        # Pastikan input dalam format yang benar
        predictions = np.array(predictions)
        labels = np.array(labels)

        if len(predictions.shape) == 1:
            predictions = predictions.reshape(-1, 1)
        if len(labels.shape) == 1:
            labels = labels.reshape(-1, 1)

        for i, genre in enumerate(classes):
            try:
                genre_preds = predictions[:, i]
                genre_labels = labels[:, i]

                # Calculate confusion matrix
                cm = confusion_matrix(genre_labels, genre_preds)

                # Extract values
                TN, FP = cm[0]
                FN, TP = cm[1]

                # Calculate metrics
                accuracy = (TP + TN) / (TP + TN + FP + FN)
                precision = TP / (TP + FP) if (TP + FP) > 0 else 0
                recall = TP / (TP + FN) if (TP + FN) > 0 else 0
                f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

                # Create plot
                plt.figure(figsize=(12, 8))

                # Main confusion matrix plot
                main_ax = plt.subplot2grid((3, 3), (0, 0), rowspan=2, colspan=2)

                # Plot heatmap
                plot_labels = [f'Non-{genre}', genre]
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                          xticklabels=plot_labels,
                          yticklabels=plot_labels,
                          ax=main_ax)

                main_ax.set_title(f'Confusion Matrix - {genre}')
                main_ax.set_ylabel('True Label')
                main_ax.set_xlabel('Predicted Label')

                # Create text box for detailed metrics
                plt.subplot2grid((3, 3), (0, 2), rowspan=3)
                plt.axis('off')

                metrics_text = [
                    f'Detailed Metrics for {genre}:\n',
                    f'\nConfusion Matrix Values:',
                    f'True Negative (TN): {TN}',
                    f'False Positive (FP): {FP}',
                    f'False Negative (FN): {FN}',
                    f'True Positive (TP): {TP}',
                    f'\nPerformance Metrics:',
                    f'Accuracy: {accuracy:.3f}',
                    f'Precision: {precision:.3f}',
                    f'Recall: {recall:.3f}',
                    f'F1 Score: {f1:.3f}',
                    f'\nAdditional Information:',
                    f'Total Samples: {len(genre_labels)}',
                    f'Positive Samples: {np.sum(genre_labels)}',
                    f'Negative Samples: {len(genre_labels) - np.sum(genre_labels)}'
                ]

                plt.text(0, 0.95, '\n'.join(metrics_text),
                        fontsize=10,
                        verticalalignment='top',
                        bbox=dict(boxstyle='round,pad=1', facecolor='white', alpha=0.8))

                # Add interpretation text
                interpretation_ax = plt.subplot2grid((3, 3), (2, 0), colspan=2)
                interpretation_ax.axis('off')

                interpretation_text = [
                    'Matrix Interpretation:',
                    f'• Model correctly identified {TN} non-{genre} movies (True Negatives)',
                    f'• Model correctly identified {TP} {genre} movies (True Positives)',
                    f'• Model incorrectly classified {FP} non-{genre} movies as {genre} (False Positives)',
                    f'• Model failed to identify {FN} {genre} movies (False Negatives)'
                ]

                interpretation_ax.text(0, 0.5, '\n'.join(interpretation_text),
                                    fontsize=9,
                                    verticalalignment='center',
                                    bbox=dict(boxstyle='round,pad=1', facecolor='lightyellow', alpha=0.3))

                plt.tight_layout()
                plot_path = Config.CM_DIR / f'confusion_matrix_{genre}.png'
                plt.savefig(plot_path, dpi=300, bbox_inches='tight')
                plt.close()

            except Exception as e:
                logging.error(f"Error plotting confusion matrix for genre {genre}: {str(e)}")
                continue

        logging.info(f"Confusion matrices saved in: {Config.CM_DIR}")

    @staticmethod
    def plot_training_history(history_data: Dict) -> None:
        """Plot and save training metrics"""
        plt.figure(figsize=(15, 5))

        # Plot Loss
        plt.subplot(1, 3, 1)
        plt.plot(history_data['epochs'], history_data['training_loss'],
                label='Training Loss', marker='o')
        plt.plot(history_data['epochs'], history_data['validation_loss'],
                label='Validation Loss', marker='o')
        plt.title('Training History - Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        # Plot Accuracy
        plt.subplot(1, 3, 2)
        plt.plot(history_data['epochs'], history_data['accuracy'],
                label='Accuracy', marker='o', color='green')
        plt.title('Training History - Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        # Plot Loss Difference
        plt.subplot(1, 3, 3)
        loss_diff = np.array(history_data['training_loss']) - np.array(history_data['validation_loss'])
        plt.plot(history_data['epochs'], loss_diff,
                label='Loss Difference', marker='o', color='red')
        plt.title('Learning Curve (Train-Val Loss)')
        plt.xlabel('Epoch')
        plt.ylabel('Difference')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plot_path = Config.PLOTS_DIR / 'training_history.png'
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        plt.close()
        logging.info(f"Saved training history plots to {plot_path}")

class ModelEvaluator:
    """Class untuk evaluasi model"""

    @staticmethod
    @error_handler
    def evaluate_model(model: torch.nn.Module,
                      val_loader: DataLoader,
                      mlb: MultiLabelBinarizer) -> Dict:
        """Evaluate model performance"""
        model.eval()
        all_preds = []
        all_labels = []
        correct_predictions = 0
        total_predictions = 0

        logging.info("Starting model evaluation...")
        log_memory("evaluation start")

        try:
            with torch.no_grad(), tqdm(val_loader, desc="Evaluating") as pbar:
                for batch in pbar:
                    input_ids = batch['input_ids'].to(Config.DEVICE)
                    attention_mask = batch['attention_mask'].to(Config.DEVICE)
                    labels = batch['labels'].cpu().numpy()

                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                    preds = torch.sigmoid(outputs.logits).cpu().numpy()
                    preds_binary = (preds > 0.5).astype(int)

                    all_preds.extend(preds_binary)
                    all_labels.extend(labels)

                    correct_predictions += np.sum(preds_binary == labels)
                    total_predictions += labels.size

                    # Update progress bar
                    pbar.set_postfix({'Accuracy': f'{(correct_predictions/total_predictions)*100:.2f}%'})

            all_preds = np.array(all_preds)
            all_labels = np.array(all_labels)
            accuracy = correct_predictions / total_predictions

            # Calculate per-genre metrics
            genre_metrics = ModelEvaluator._calculate_genre_metrics(
                all_preds, all_labels, mlb.classes_
            )

            # Calculate macro metrics
            macro_metrics = ModelEvaluator._calculate_macro_metrics(genre_metrics)

            # Save metrics
            evaluation_metrics = {
                'overall': macro_metrics,
                'per_genre': genre_metrics
            }

            metrics_file = Config.METRICS_DIR / 'evaluation_metrics.json'
            with open(metrics_file, 'w', encoding='utf-8') as f:
                json.dump(evaluation_metrics, f, indent=4, ensure_ascii=False)

            # Plot confusion matrices
            Visualization.plot_confusion_matrices(all_preds, all_labels, mlb.classes_)

            log_memory("evaluation end")

            return {
                'accuracy': float(accuracy),
                'macro_f1': float(macro_metrics['macro_f1']),
                'genre_metrics': genre_metrics
            }

        except Exception as e:
            logging.error(f"Error during model evaluation: {str(e)}")
            raise

    @staticmethod
    def _calculate_genre_metrics(predictions: np.ndarray,
                               labels: np.ndarray,
                               classes: List[str]) -> Dict:
        """Calculate metrics for each genre"""
        genre_metrics = {}
        logging.info("\nPer-genre Performance Metrics:")

        for i, genre in enumerate(classes):
            genre_preds = predictions[:, i]
            genre_labels = labels[:, i]

            metrics = {
                'accuracy': float(np.mean(genre_preds == genre_labels)),
                'f1_score': float(f1_score(genre_labels, genre_preds, zero_division=0)),
                'precision': float(precision_score(genre_labels, genre_preds, zero_division=0)),
                'recall': float(recall_score(genre_labels, genre_preds, zero_division=0))
            }

            genre_metrics[genre] = metrics
            logging.info(f"\nMetrics for {genre}:")
            for metric_name, value in metrics.items():
                logging.info(f"{metric_name.capitalize()}: {value:.4f}")

        return genre_metrics

    @staticmethod
    def _calculate_macro_metrics(genre_metrics: Dict) -> Dict:
        """Calculate macro-averaged metrics"""
        return {
            'accuracy': np.mean([metrics['accuracy'] for metrics in genre_metrics.values()]),
            'macro_f1': np.mean([metrics['f1_score'] for metrics in genre_metrics.values()]),
            'macro_precision': np.mean([metrics['precision'] for metrics in genre_metrics.values()]),
            'macro_recall': np.mean([metrics['recall'] for metrics in genre_metrics.values()])
        }

In [6]:
# BAGIAN KEEMPAT - Training dan Hyperparameter Optimization

class ModelTrainer:
    @staticmethod
    @error_handler
    def train_model(sample_size: Optional[int] = None) -> Tuple:
        Config.SAMPLE_SIZE = sample_size
        logging.info("Starting model training")
        log_memory("training start")

        try:
            # Load dan preprocess data
            df = DataProcessor.load_and_preprocess_data(Config.DATA_PATH, Config.SAMPLE_SIZE)
            logging.info(f"\nDataset statistics:")
            logging.info(f"Total samples after preprocessing: {len(df)}")

            # Prepare data
            mlb = MultiLabelBinarizer()
            X_train, X_test, y_train, y_test = DataProcessor.prepare_data(df, mlb)

            # Log genre distribution
            genre_labels = mlb.fit_transform(df['genre'])
            genre_counts = genre_labels.sum(axis=0)
            for genre, count in zip(mlb.classes_, genre_counts):
                logging.info(f"Genre '{genre}': {count} samples")

            logging.info(f"\nTraining set size: {len(X_train)}")
            logging.info(f"Testing set size: {len(X_test)}")

            # Setup model dan data loaders
            with ModelManager(*ModelSetup.setup_model_and_tokenizer(len(mlb.classes_))) as (model, tokenizer):
                train_loader, val_loader = ModelSetup.setup_dataloaders(
                    X_train, X_test, y_train, y_test,
                    tokenizer, Config.MODEL_PARAMS['BATCH_SIZE']
                )

                # Training loop
                best_val_loss = float('inf')
                best_accuracy = 0.0
                patience_counter = 0
                training_losses = []
                validation_losses = []
                accuracies = []

                optimizer = torch.optim.AdamW(
                    model.parameters(),
                    lr=Config.MODEL_PARAMS['LEARNING_RATE'],
                    weight_decay=Config.MODEL_PARAMS['WEIGHT_DECAY']
                )

                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                    optimizer, mode='min', factor=0.5, patience=2, verbose=True
                )

                # Training epochs with progress bar
                epochs_range = tqdm(range(Config.MODEL_PARAMS['EPOCHS']), desc="Training Progress",
                                  position=0, leave=True)
                for epoch in epochs_range:
                    try:
                        # Training phase
                        model.train()
                        epoch_metrics = ModelTrainer._train_epoch(
                            model, train_loader, optimizer,
                            epoch + 1
                        )
                        training_losses.append(epoch_metrics['train_loss'])

                        # Evaluation phase
                        validation_metrics = ModelEvaluator.evaluate_model(model, val_loader, mlb)
                        current_accuracy = validation_metrics['accuracy']
                        accuracies.append(current_accuracy)

                        # Validation loss calculation
                        avg_val_loss = ModelTrainer._calculate_validation_loss(
                            model, val_loader
                        )
                        validation_losses.append(avg_val_loss)

                        # Update progress bar
                        epochs_range.set_postfix({
                            'Train Loss': f"{epoch_metrics['train_loss']:.4f}",
                            'Val Loss': f"{avg_val_loss:.4f}",
                            'Accuracy': f"{current_accuracy:.4f}"
                        })

                        # Model improvement check
                        model_improved = ModelTrainer._check_model_improvement(
                            model, tokenizer, current_accuracy, avg_val_loss,
                            best_accuracy, best_val_loss
                        )

                        if model_improved:
                            best_accuracy = max(best_accuracy, current_accuracy)
                            best_val_loss = min(best_val_loss, avg_val_loss)
                            patience_counter = 0
                        else:
                            patience_counter += 1

                        # Early stopping check
                        if patience_counter >= Config.MODEL_PARAMS['PATIENCE']:
                            logging.info(f"\nEarly stopping triggered after {epoch + 1} epochs")
                            break

                        scheduler.step(avg_val_loss)
                        logging.info(f"Learning rate: {optimizer.param_groups[0]['lr']}")

                        # Clear GPU cache periodically
                        if torch.cuda.is_available() and (epoch + 1) % 5 == 0:
                            torch.cuda.empty_cache()
                            gc.collect()

                    except Exception as e:
                        logging.error(f"Error during epoch {epoch + 1}: {str(e)}")
                        raise

                # Save training history
                ModelTrainer._save_training_history(
                    training_losses, validation_losses, accuracies,
                    best_accuracy, best_val_loss, df, mlb, genre_counts
                )

                return model, tokenizer, mlb

        except Exception as e:
            logging.error(f"Error in training: {str(e)}")
            raise
        finally:
            # Cleanup
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

    @staticmethod
    def _train_epoch(model: torch.nn.Module,
                    train_loader: DataLoader,
                    optimizer: torch.optim.Optimizer,
                    epoch: int) -> Dict:
        """Train model for one epoch"""
        total_loss = 0
        steps = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}",
                          position=1, leave=False)

        for batch in progress_bar:
            try:
                optimizer.zero_grad()

                if np.random.random() < Config.MODEL_PARAMS['MIXUP_PROB']:
                    batch = DataAugmentation.apply_mixup(batch)

                input_ids = batch['input_ids'].to(Config.DEVICE)
                attention_mask = batch['attention_mask'].to(Config.DEVICE)
                labels = batch['labels'].to(Config.DEVICE)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = LossFunctions.label_smoothing_loss(
                    outputs.logits, labels, Config.MODEL_PARAMS['SMOOTHING']
                )

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                total_loss += loss.item()
                steps += 1
                progress_bar.set_postfix({'training_loss': f'{loss.item():.4f}'})

            except RuntimeError as e:
                if "out of memory" in str(e):
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    logging.error("GPU out of memory during training. Try reducing batch size.")
                raise

        return {'train_loss': total_loss / steps}

    @staticmethod
    def _calculate_validation_loss(model: torch.nn.Module,
                                 val_loader: DataLoader) -> float:
        """Calculate validation loss"""
        model.eval()
        total_loss = 0
        steps = 0

        with torch.no_grad():
            for batch in val_loader:
                try:
                    input_ids = batch['input_ids'].to(Config.DEVICE)
                    attention_mask = batch['attention_mask'].to(Config.DEVICE)
                    labels = batch['labels'].to(Config.DEVICE)

                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                    loss = LossFunctions.focal_loss(outputs.logits, labels)
                    total_loss += loss.item()
                    steps += 1

                except RuntimeError as e:
                    if "out of memory" in str(e):
                        if torch.cuda.is_available():
                            torch.cuda.empty_cache()
                        logging.error("GPU out of memory during validation. Try reducing batch size.")
                    raise

        return total_loss / steps

    @staticmethod
    def _check_model_improvement(model: torch.nn.Module,
                               tokenizer,
                               current_accuracy: float,
                               current_loss: float,
                               best_accuracy: float,
                               best_loss: float) -> bool:
        """Check if model improved and save if necessary"""
        improved = False

        try:
            if current_accuracy > best_accuracy:
                logging.info(f"New best accuracy: {current_accuracy:.4f}")
                model.save_pretrained(str(Config.MODEL_BEST_ACC))  # Convert to string for Colab
                tokenizer.save_pretrained(str(Config.TOKENIZER_BEST_ACC))
                improved = True

            if current_loss < best_loss:
                logging.info(f"New best loss: {current_loss:.4f}")
                model.save_pretrained(str(Config.MODEL_BEST_LOSS))
                tokenizer.save_pretrained(str(Config.TOKENIZER_BEST_LOSS))
                improved = True

        except Exception as e:
            logging.error(f"Error saving model: {str(e)}")
            raise

        return improved

    @staticmethod
    def _save_training_history(training_losses: List[float],
                             validation_losses: List[float],
                             accuracies: List[float],
                             best_accuracy: float,
                             best_val_loss: float,
                             df: pd.DataFrame,
                             mlb: MultiLabelBinarizer,
                             genre_counts: np.ndarray) -> None:
        """Save training history and plot results"""
        try:
            history_data = {
                'model_info': {
                    'classes': mlb.classes_.tolist(),
                    'total_samples': len(df),
                    'genre_distribution': {
                        genre: int(count) for genre, count in zip(mlb.classes_, genre_counts)
                    }
                },
                'training_config': {
                    'batch_size': Config.MODEL_PARAMS['BATCH_SIZE'],
                    'learning_rate': Config.MODEL_PARAMS['LEARNING_RATE'],
                    'max_length': Config.MODEL_PARAMS['MAX_LENGTH'],
                    'weight_decay': Config.MODEL_PARAMS['WEIGHT_DECAY'],
                    'early_stopping': Config.MODEL_PARAMS['PATIENCE'],
                    'mixup_prob': Config.MODEL_PARAMS['MIXUP_PROB'],
                    'train_split': 1-Config.MODEL_PARAMS['TEST_SIZE'],
                    'test_split': Config.MODEL_PARAMS['TEST_SIZE']
                },
                'training_history': {
                    'epochs': list(range(1, len(training_losses) + 1)),
                    'training_loss': [float(loss) for loss in training_losses],
                    'validation_loss': [float(loss) for loss in validation_losses],
                    'accuracy': [float(acc) for acc in accuracies],
                    'best_accuracy': float(best_accuracy),
                    'best_val_loss': float(best_val_loss)
                }
            }

            # Save history to JSON
            history_file = Config.METRICS_DIR / 'training_history.json'
            with open(history_file, 'w', encoding='utf-8') as f:
                json.dump(history_data, f, indent=4, ensure_ascii=False)

            # Plot training history
            Visualization.plot_training_history(history_data['training_history'])
            logging.info("Training history saved successfully")

        except Exception as e:
            logging.error(f"Error saving training history: {str(e)}")
            raise


class HyperparameterOptimizer:
    @staticmethod
    @error_handler
    def objective(trial: Trial, df: pd.DataFrame, mlb: MultiLabelBinarizer) -> float:
        """Objective function untuk Optuna optimization"""
        try:
            # Get trial parameters
            params = HyperparameterOptimizer._get_trial_parameters(trial)

            # Prepare data
            X_train, X_test, y_train, y_test = DataProcessor.prepare_data(df, mlb)

            # Setup model dan data loaders
            with ModelManager(*ModelSetup.setup_model_and_tokenizer(len(mlb.classes_))) as (model, tokenizer):
                train_loader, val_loader = ModelSetup.setup_dataloaders(
                    X_train, X_test, y_train, y_test,
                    tokenizer, params['batch_size']
                )

                # Training loop singkat untuk optimasi
                best_val_metrics = HyperparameterOptimizer._train_trial(
                    trial, model, train_loader, val_loader, mlb, params
                )

                # Clear GPU cache after each trial
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    gc.collect()

            return best_val_metrics['macro_f1']

        except Exception as e:
            logging.error(f"Error in optimization objective: {str(e)}")
            raise

    @staticmethod
    def _get_trial_parameters(trial: Trial) -> Dict:
        """Get parameters for trial"""
        params = {}
        try:
            # Batch size dari list nilai diskrit
            params['batch_size'] = trial.suggest_categorical('batch_size',
                Config.OPTIM_PARAMS['batch_size'])

            # Learning rate dari list nilai diskrit
            params['learning_rate'] = trial.suggest_categorical('learning_rate',
                Config.OPTIM_PARAMS['learning_rate'])

            # Weight decay dari list nilai diskrit
            params['weight_decay'] = trial.suggest_categorical('weight_decay',
                Config.OPTIM_PARAMS['weight_decay'])

            # Mixup probability dari list nilai diskrit
            params['mixup_prob'] = trial.suggest_categorical('mixup_prob',
                Config.OPTIM_PARAMS['mixup_prob'])

            # Smoothing dari list nilai diskrit
            params['smoothing'] = trial.suggest_categorical('smoothing',
                Config.OPTIM_PARAMS['smoothing'])

            logging.info(f"Trial parameter set: {params}")

        except Exception as e:
            logging.error(f"Error getting trial parameters: {str(e)}")
            raise
        return params

    @staticmethod
    def _train_trial(trial: Trial,
                    model: torch.nn.Module,
                    train_loader: DataLoader,
                    val_loader: DataLoader,
                    mlb: MultiLabelBinarizer,
                    params: Dict) -> Dict:
        """Train model for one trial"""
        try:
            optimizer = torch.optim.AdamW(
                model.parameters(),
                lr=params['learning_rate'],
                weight_decay=params['weight_decay']
            )

            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=2, verbose=True
            )

            best_val_metrics = None

            for epoch in range(3):  # Reduced epochs for optimization
                # Training
                model.train()
                total_loss = 0
                steps = 0

                progress_bar = tqdm(train_loader,
                                  desc=f"Epoch {epoch+1}/3",
                                  position=0,
                                  leave=False)

                for batch in progress_bar:
                    try:
                        optimizer.zero_grad()

                        if np.random.random() < params['mixup_prob']:
                            batch = DataAugmentation.apply_mixup(batch)

                        input_ids = batch['input_ids'].to(Config.DEVICE)
                        attention_mask = batch['attention_mask'].to(Config.DEVICE)
                        labels = batch['labels'].to(Config.DEVICE)

                        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                        loss = LossFunctions.label_smoothing_loss(
                            outputs.logits, labels, params['smoothing']
                        )

                        loss.backward()
                        optimizer.step()

                        total_loss += loss.item()
                        steps += 1

                        # Update progress bar
                        progress_bar.set_postfix({
                            'loss': f'{loss.item():.4f}',
                            'avg_loss': f'{(total_loss/steps):.4f}'
                        })

                    except RuntimeError as e:
                        if "out of memory" in str(e):
                            if torch.cuda.is_available():
                                torch.cuda.empty_cache()
                            logging.error("GPU OOM in trial. Trying to recover...")
                            continue
                        raise

                avg_loss = total_loss / steps

                # Evaluation
                metrics = ModelEvaluator.evaluate_model(model, val_loader, mlb)
                current_f1 = metrics['macro_f1']

                logging.info(f"Trial {trial.number}, Epoch {epoch+1}: "
                           f"Loss = {avg_loss:.4f}, F1 = {current_f1:.4f}")

                if best_val_metrics is None or current_f1 > best_val_metrics['macro_f1']:
                    best_val_metrics = metrics

                scheduler.step(metrics['macro_f1'])

                # Report intermediate value
                trial.report(metrics['macro_f1'], epoch)

                # Handle pruning based on the intermediate value
                if trial.should_prune():
                    raise optuna.TrialPruned()

                # Clear GPU cache
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    gc.collect()

            return best_val_metrics

        except Exception as e:
            logging.error(f"Error in trial training: {str(e)}")
            raise

    @staticmethod
    def run_optimization(df: pd.DataFrame,
                        mlb: MultiLabelBinarizer,
                        n_trials: int = 30) -> Dict:

        try:
            study = optuna.create_study(
                direction="maximize",
                sampler=optuna.samplers.TPESampler(seed=42),
                pruner=optuna.pruners.MedianPruner()
            )

            objective_func = lambda trial: HyperparameterOptimizer.objective(trial, df, mlb)

            logging.info("Starting hyperparameter optimization...")
            study.optimize(objective_func, n_trials=n_trials,
                         callbacks=[lambda study, trial: gc.collect()])

            # Log results
            logging.info("\nHyperparameter Optimization Results:")
            logging.info(f"Best trial number: {study.best_trial.number}")
            logging.info(f"Best F1-score: {study.best_trial.value:.4f}")
            logging.info("\nBest hyperparameters:")
            for param, value in study.best_trial.params.items():
                logging.info(f"{param}: {value}")

            # Save study results
            results_file = Config.METRICS_DIR / 'optuna_results.json'
            results = {
                'best_trial': {
                    'number': study.best_trial.number,
                    'value': study.best_trial.value,
                    'params': study.best_trial.params
                },
                'all_trials': [
                    {
                        'number': trial.number,
                        'value': trial.value,
                        'params': trial.params
                    }
                    for trial in study.trials if trial.value is not None
                ]
            }

            with open(results_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=4, ensure_ascii=False)

            # Save visualizations
            try:
                # Optimization history plot
                fig1 = optuna.visualization.plot_optimization_history(study)
                fig1.write_image(str(Config.PLOTS_DIR / "optuna_optimization_history.png"))

                # Parameter importance plot
                fig2 = optuna.visualization.plot_param_importances(study)
                fig2.write_image(str(Config.PLOTS_DIR / "optuna_param_importances.png"))

                # Parameter relationships plot
                fig3 = optuna.visualization.plot_parallel_coordinate(study)
                fig3.write_image(str(Config.PLOTS_DIR / "optuna_param_relationships.png"))

            except Exception as e:
                logging.warning(f"Could not create optimization plots: {str(e)}")

            return study.best_trial.params

        except Exception as e:
            logging.error(f"Error during optimization: {str(e)}")
            raise
        finally:
            # Clean up
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

# Fungsi get_args untuk Colab
class Args:
    def __init__(self):
        # Training parameters
        self.sample_size = None  # Number of samples to use (default: use all data)
        self.epochs = Config.MODEL_PARAMS['EPOCHS']
        self.batch_size = Config.MODEL_PARAMS['BATCH_SIZE']
        self.learning_rate = Config.MODEL_PARAMS['LEARNING_RATE']
        self.max_length = Config.MODEL_PARAMS['MAX_LENGTH']

        # Model configuration
        self.test_size = Config.MODEL_PARAMS['TEST_SIZE']
        self.weight_decay = Config.MODEL_PARAMS['WEIGHT_DECAY']
        self.mixup_prob = Config.MODEL_PARAMS['MIXUP_PROB']
        self.patience = Config.MODEL_PARAMS['PATIENCE']

        # System configuration
        self.output_dir = None
        self.no_cuda = False
        self.seed = 42

        # Label Smoothing
        self.smoothing = Config.MODEL_PARAMS['SMOOTHING']

        # Optuna specific
        self.n_trials = 20

# Modifikasi fungsi get_args
def get_args():
    return Args()

def main(args: Args) -> None:
    """Main function"""
    try:
        # Check environment first
        if not check_environment():
            raise RuntimeError("Environment check failed!")

        # Update configuration
        Config.MODEL_PARAMS.update({
            'EPOCHS': args.epochs,
            'BATCH_SIZE': args.batch_size,
            'LEARNING_RATE': args.learning_rate,
            'MAX_LENGTH': args.max_length,
            'TEST_SIZE': args.test_size,
            'WEIGHT_DECAY': args.weight_decay,
            'MIXUP_PROB': args.mixup_prob,
            'PATIENCE': args.patience,
            'SMOOTHING': args.smoothing
        })

        if args.no_cuda:
            Config.DEVICE = torch.device('cpu')
            logging.info("CUDA disabled by user")

        # Initialize logging dan experiment info
        log_system_info()

        logging.info("Starting movie genre classification with hyperparameter optimization")
        logging.info(f"Using device: {Config.DEVICE}")

        # Log initial configuration
        logging.info("\nInitial Configuration:")
        logging.info(f"Sample Size: {Config.SAMPLE_SIZE if Config.SAMPLE_SIZE else 'Full Dataset'}")
        for param, value in Config.MODEL_PARAMS.items():
            logging.info(f"{param}: {value}")

        # Load and preprocess data
        logging.info("\nLoading and preprocessing data...")
        df = DataProcessor.load_and_preprocess_data(Config.DATA_PATH, Config.SAMPLE_SIZE)
        mlb = MultiLabelBinarizer()

        # Run hyperparameter optimization
        logging.info("\nStarting hyperparameter optimization...")
        best_params = HyperparameterOptimizer.run_optimization(df, mlb, args.n_trials)

        # Update configuration with best parameters
        logging.info("\nBest Hyperparameters found:")
        for param, value in best_params.items():
            logging.info(f"{param}: {value}")
            if param in Config.MODEL_PARAMS:
                Config.MODEL_PARAMS[param] = value

        # Train final model with best parameters
        logging.info("\nTraining final model with optimized parameters...")
        model, tokenizer, mlb = ModelTrainer.train_model(Config.SAMPLE_SIZE)

        # Test on a sample
        logging.info("\nTesting model on a sample...")
        df_sample = DataProcessor.load_and_preprocess_data(Config.DATA_PATH, sample_size=1)
        sample_text = df_sample['sinopsis'].iloc[0]

        model.eval()
        inputs = tokenizer(
            sample_text,
            return_tensors='pt',
            max_length=Config.MODEL_PARAMS['MAX_LENGTH'],
            padding='max_length',
            truncation=True
        )

        input_ids = inputs['input_ids'].to(Config.DEVICE)
        attention_mask = inputs['attention_mask'].to(Config.DEVICE)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probs = torch.sigmoid(outputs.logits)

        predictions = []
        for idx, prob in enumerate(probs[0]):
            if prob > 0.5:
                predictions.append({
                    'genre': mlb.classes_[idx],
                    'probability': float(prob.item())
                })

        predictions.sort(key=lambda x: x['probability'], reverse=True)

        # Log prediction results
        logging.info("\nSample prediction results:")
        logging.info(f"Sample text: {sample_text[:100]}...")
        for pred in predictions:
            logging.info(f"{pred['genre']}: {pred['probability']:.4f}")

        # Save final configuration
        final_config = {
            'hyperparameters': best_params,
            'model_info': {
                'num_classes': len(mlb.classes_),
                'classes': mlb.classes_.tolist()
            },
            'training_info': {
                'device': str(Config.DEVICE),
                'final_sample_size': len(df),
                'optimization_trials': args.n_trials
            }
        }

        with open(Config.EXPERIMENT_DIR / 'final_configuration.json', 'w', encoding='utf-8') as f:
            json.dump(final_config, f, indent=4, ensure_ascii=False)

        logging.info("\nTraining completed successfully!")
        logging.info(f"All results and models saved in: {Config.EXPERIMENT_DIR}")

    except KeyboardInterrupt:
        logging.info("\nTraining interrupted by user")
        raise
    except Exception as e:
        logging.error(f"\nError during execution: {str(e)}")
        raise
    finally:
        logging.info("\nCleaning up resources...")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

# Entry point for Colab
if __name__ == "__main__":
    # Set random seeds for reproducibility
    args = get_args()

    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

    try:
        main(args)
    except KeyboardInterrupt:
        logging.info("\nTraining interrupted by user")
    except Exception as e:
        logging.error(f"Training failed: {str(e)}")
        raise
    finally:
        logging.info("Cleaning up resources...")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()


Files in dataset directory:
- final_combined_movies_5genres.csv
GPU tersedia: Tesla T4
GPU Memory: 15.83 GB
2025-02-19 08:40:04,613 - INFO - System Information:
2025-02-19 08:40:04,614 - INFO - Python Version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
2025-02-19 08:40:04,616 - INFO - CPU Count: 4
2025-02-19 08:40:04,617 - INFO - Initial Memory Usage: 635.94 MB
2025-02-19 08:40:04,618 - INFO - GPU Device: Tesla T4
2025-02-19 08:40:04,619 - INFO - GPU Memory Total: 15.83 GB
2025-02-19 08:40:04,619 - INFO - CUDA Version: 12.1
2025-02-19 08:40:04,620 - INFO - Starting movie genre classification with hyperparameter optimization
2025-02-19 08:40:04,621 - INFO - Using device: cuda
2025-02-19 08:40:04,622 - INFO - 
Initial Configuration:
2025-02-19 08:40:04,623 - INFO - Sample Size: Full Dataset
2025-02-19 08:40:04,624 - INFO - EPOCHS: 100
2025-02-19 08:40:04,624 - INFO - BATCH_SIZE: 10
2025-02-19 08:40:04,625 - INFO - LEARNING_RATE: 1e-05
2025-02-19 08:40:04,626 - INFO - MAX_LENGTH:

100%|██████████| 1738/1738 [00:00<00:00, 14951.85it/s]

2025-02-19 08:40:04,822 - INFO - Memory usage after preprocessing: 640.25 MB
2025-02-19 08:40:04,823 - INFO - 
Starting hyperparameter optimization...



[I 2025-02-19 08:40:04,824] A new study created in memory with name: no-name-9457c34d-6e47-491f-8672-3359143159fe


2025-02-19 08:40:04,825 - INFO - Starting hyperparameter optimization...
2025-02-19 08:40:04,828 - INFO - Trial parameter set: {'batch_size': 4, 'learning_rate': 3e-06, 'weight_decay': 0.02, 'mixup_prob': 0.3, 'smoothing': 0.15}
2025-02-19 08:40:04,834 - INFO - Setting up model and tokenizer...


...

Training Progress:  11%|█         | 11/100 [30:46<4:09:02, 167.89s/it, Train Loss=1.1950, Val Loss=0.0529, Accuracy=0.6989]


2025-02-19 11:18:09,902 - INFO - Saved training history plots to /kaggle/working/logs/experiments/20250219_084004/plots/training_history.png
2025-02-19 11:18:09,903 - INFO - Training history saved successfully
2025-02-19 11:18:12,454 - INFO - 
Testing model on a sample...
2025-02-19 11:18:12,473 - INFO - Loading and preprocessing data...
2025-02-19 11:18:12,474 - INFO - Memory usage after start: 4059.28 MB
2025-02-19 11:18:12,487 - INFO - Successfully loaded data using utf-8 encoding
2025-02-19 11:18:12,488 - INFO - Memory usage after data loading: 4059.28 MB
2025-02-19 11:18:12,489 - INFO - Taking sample of 1 from 1738 total samples
2025-02-19 11:18:12,490 - INFO - 
Sample data:
2025-02-19 11:18:12,491 - INFO - 
Sample 1:
2025-02-19 11:18:12,492 - INFO - Synopsis: Setelah kematian yang tampak, Siena mampu melihat tanda-tanda bahwa orang-orang akan meninggal. Namu...
2025-02-19 11:18:12,493 - INFO - Genre: Horor
2025-02-19 11:18:12,494 - INFO - 
Preprocessing text data...


100%|██████████| 1/1 [00:00<00:00, 2542.00it/s]

2025-02-19 11:18:12,501 - INFO - Memory usage after preprocessing: 4059.28 MB
2025-02-19 11:18:12,552 - INFO - 
Sample prediction results:
2025-02-19 11:18:12,552 - INFO - Sample text: setelah kematian yang tampak siena mampu melihat tanda tanda bahwa orang orang akan meninggal namun ...
2025-02-19 11:18:12,553 - INFO - Horor: 0.9210
2025-02-19 11:18:12,555 - INFO - Drama: 0.5230
2025-02-19 11:18:12,556 - INFO - 
Training completed successfully!
2025-02-19 11:18:12,556 - INFO - All results and models saved in: /kaggle/working/logs/experiments/20250219_084004
2025-02-19 11:18:12,557 - INFO - 
Cleaning up resources...


2025-02-19 11:18:13,863 - INFO - Cleaning up resources...


In [7]:
!zip -r folder.zip /kaggle/working/logs/experiments

  adding: kaggle/working/logs/experiments/ (stored 0%)
  adding: kaggle/working/logs/experiments/20250219_084004/ (stored 0%)
  adding: kaggle/working/logs/experiments/20250219_084004/metrics/ (stored 0%)
  adding: kaggle/working/logs/experiments/20250219_084004/metrics/training_history.json (deflated 66%)
  adding: kaggle/working/logs/experiments/20250219_084004/metrics/evaluation_metrics.json (deflated 69%)
  adding: kaggle/working/logs/experiments/20250219_084004/metrics/optuna_results.json (deflated 90%)
  adding: kaggle/working/logs/experiments/20250219_084004/final_configuration.json (deflated 52%)
  adding: kaggle/working/logs/experiments/20250219_084004/training.log (deflated 86%)
  adding: kaggle/working/logs/experiments/20250219_084004/tokenizer/ (stored 0%)
  adding: kaggle/working/logs/experiments/20250219_084004/tokenizer/best_loss/ (stored 0%)
  adding: kaggle/working/logs/experiments/20250219_084004/tokenizer/best_loss/tokenizer.json (deflated 71%)
  adding: kaggle/worki

In [8]:
from IPython.display import FileLink
FileLink(r'folder.zip')

/kaggle/working/folder.zip